ADP data by Steven Cunningham 06/16/17

This code is intened for take SonTek ADP data
- First determine seafloor by signal amplitude file ".a#". The signal should slowly decay through water, when decay is
  interupted with amplitude spike, then the signal is bouncing back from the floor. 
- Compare one bin to the previous until the next bin is larger than previous.
- Number of good profiles will be used on velocity data. 

In [137]:
import pandas as pd
import numpy as np
import glob
import matplotlib.pyplot as plt
from scipy import stats
import scipy
import csv
import os
from os import listdir

In [138]:
path = 'C:/Users/steve/Desktop/Thesis/ADP files/10.13.16'
file_group = 'SWC0610131207'
os.chdir(path)
sig_amp = '%s.a2'% file_group
VE_file = '%s.ve'% file_group
VN_file = '%s.vn'% file_group
VU_file = '%s.vu'% file_group
save_name=(sig_amp[:-3]) # save file name string
save_file_name = (path[-8:])
df = pd.read_csv(sig_amp, delimiter='\s+', header=None)
df.head()


,0,1,2,3,4,5,6,7,8,9,...,13,14,15,16,17,18,19,20,21,22
0,1,154,116,118,113,106,93,86,78,71,...,138,120,95,67,41,30,25,24,23,23
1,2,140,128,118,112,103,94,87,80,71,...,130,113,113,93,56,39,30,26,24,23
2,3,137,128,119,110,104,95,84,77,70,...,117,126,112,69,44,33,28,25,24,24
3,4,146,130,118,111,104,96,88,79,71,...,124,96,86,50,34,29,26,24,24,23
4,5,154,135,122,113,108,97,88,80,75,...,89,114,148,93,57,41,31,26,24,24


In [139]:
mean = np.mean(df, axis = 0)[1:] # average profiles across a depth bin
mean2 = pd.Series.to_frame(mean, 'average_profile') # transform to dataframe 
mean2["next_profile"] = mean2["average_profile"].shift(-1) # Shift down to commpare difference 
mean2["Nartural_decay"] = mean2["average_profile"] > mean2["next_profile"] # check if the next bin is greater than previous 
mean2["True_profile"] = mean2["Nartural_decay"].cumsum() # gives values to groups of true/false
mean2["Number"] = np.arange(23)[1:] # make a range of values 
mean2["good_profile"] = mean2["True_profile"] == mean2["Number"] # see if values assigned to true/false match number order  
print(mean2)

how_many=sum(mean2["good_profile"]) # how many good profiles are there. This will be used on velocity data 
mean3=mean2["average_profile"][:how_many+1] # get profile values for good bins. Add 1 because the .shift 


    average_profile  next_profile Nartural_decay  True_profile  Number  \
1           141.125       127.875           True             1       1   
2           127.875       118.625           True             2       2   
3           118.625       111.250           True             3       3   
4           111.250       104.000           True             4       4   
5           104.000        95.625           True             5       5   
6            95.625        88.375           True             6       6   
7            88.375        81.250           True             7       7   
8            81.250        73.750           True             8       8   
9            73.750        68.500           True             9       9   
10           68.500        76.250          False             9      10   
11           76.250       110.625          False             9      11   
12          110.625       114.500          False             9      12   
13          114.500       111.625     

Now calculate current speed by averaging velocities and direction 

In [140]:


VE = pd.read_csv(VE_file, delimiter='\s+', header=None)
VE.head() #depth = axis 0 = profile, axis 1 = depth
VN = pd.read_csv(VN_file, delimiter='\s+', header=None)
VU = pd.read_csv(VU_file, delimiter='\s+', header=None)
VU.head() #depth = axis 0 = profile, axis 1 = depth

,0,1,2,3,4,5,6,7,8,9,...,13,14,15,16,17,18,19,20,21,22
0,1,-0.6,-0.7,-0.6,-0.5,-1.0,-0.4,0.0,-0.1,-0.6,...,-0.5,0.0,0.3,1.4,2.2,4.1,9.1,27.8,37.7,37.4
1,2,-0.8,0.4,0.0,-0.5,-0.7,-0.6,0.5,-1.0,-0.3,...,1.2,0.6,0.4,1.1,2.9,12.8,28.4,43.2,62.0,71.7
2,3,0.4,0.8,0.9,0.7,0.4,0.0,0.4,-0.3,0.2,...,0.9,1.9,1.4,2.9,9.1,21.3,35.7,58.0,85.5,91.3
3,4,0.6,0.2,0.5,0.8,0.8,0.4,-0.1,0.0,0.3,...,0.3,0.9,1.0,4.6,12.4,27.1,49.7,71.9,90.4,104.7
4,5,0.0,1.0,0.8,0.6,0.0,0.8,0.5,0.8,0.0,...,1.1,1.9,0.0,2.2,7.2,19.5,37.0,57.1,84.1,104.9


In [141]:
VE=VE.iloc[:,1:how_many +1] # use how_many from aplitude to index good velocity profiles, add one because of 0 index
VE.head()
VN=VN.iloc[:,1:how_many +1] # use how_many from aplitude to index good velocity profiles, add one because of 0 index
VU = VU.iloc[:,1:how_many +1]
VU.head()

,1,2,3,4,5,6,7,8,9
0,-0.6,-0.7,-0.6,-0.5,-1.0,-0.4,0.0,-0.1,-0.6
1,-0.8,0.4,0.0,-0.5,-0.7,-0.6,0.5,-1.0,-0.3
2,0.4,0.8,0.9,0.7,0.4,0.0,0.4,-0.3,0.2
3,0.6,0.2,0.5,0.8,0.8,0.4,-0.1,0.0,0.3
4,0.0,1.0,0.8,0.6,0.0,0.8,0.5,0.8,0.0


In [142]:
# average across same depth and round to .001
VEV= np.array(np.round(np.mean(VE, axis = 0), 3)) 
VNV = np.array(np.round(np.mean(VN, axis = 0), 3))
VUV = np.array(np.round(np.mean(VU, axis = 0), 3))
NE_vector = (VEV**2 + VNV**2)**.5
print(NE_vector)
print(VNV, VEV)
print(VUV)

[ 1.3253939   0.53001226  1.74371557  1.30152564  0.9303897   0.54301842
  1.09114344  0.85180338  0.63544787]
[ 1.275  0.512  1.7    1.3    0.8    0.338  0.187  0.212  0.375] [-0.362  0.137  0.388 -0.063  0.475  0.425 -1.075 -0.825  0.513]
[ 0.162  0.525  0.475  0.438  0.2    0.288  0.462  0.262 -0.025]


In [143]:
North_sign = VNV > 0
East_sign = VEV >0

In [144]:
VNV = np.array(VNV) # change from DataFrame to np.array
VEV = np.array(VEV)
resultsN = []
for i in (VNV):    #make lists of the direction from positive or negative velocities 
    if i > 0:
        resultsN.extend("N")
    else:
        resultsN.extend("S")
print(resultsN)

['N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N']


In [145]:
resultsE = []
for i in (VEV):
    if i > 0:
        resultsE.extend("E")
    else:
        resultsE.extend("W")
print(resultsE)

['W', 'E', 'E', 'W', 'E', 'E', 'W', 'W', 'E']


In [146]:

depth = np.arange(1,how_many+1)
depth = depth*.7

total_velocity = pd.DataFrame(NE_vector, columns = ["Velocity [cm/s]"])
total_velocity['ADP_file'] = sig_amp[:-3]
total_velocity['Depth (m)'] = depth
total_velocity['Direction1'] = resultsN
total_velocity['Direction2'] = resultsE
total_velocity['verticle_velocity [cm/s]'] = VUV
total_velocity = total_velocity[['ADP_file', 'Velocity [cm/s]', 'verticle_velocity [cm/s]', 'Depth (m)', 'Direction1', 'Direction2']] 

print(total_velocity)
print(save_name)

        ADP_file  Velocity [cm/s]  verticle_velocity [cm/s]  Depth (m)  \
0  SWC0610131207         1.325394                     0.162        0.7   
1  SWC0610131207         0.530012                     0.525        1.4   
2  SWC0610131207         1.743716                     0.475        2.1   
3  SWC0610131207         1.301526                     0.438        2.8   
4  SWC0610131207         0.930390                     0.200        3.5   
5  SWC0610131207         0.543018                     0.288        4.2   
6  SWC0610131207         1.091143                     0.462        4.9   
7  SWC0610131207         0.851803                     0.262        5.6   
8  SWC0610131207         0.635448                    -0.025        6.3   

  Direction1 Direction2  
0          N          W  
1          N          E  
2          N          E  
3          N          W  
4          N          E  
5          N          E  
6          N          W  
7          N          W  
8          N          E  

In [147]:
filename = "%s_%s.csv" % (path[-8:], save_name[3:])

total_velocity.to_csv(filename) #save copy in path directory 
